In [83]:
import os
from symfit import variables, Parameter, Model, Fit, exp
from symfit.contrib.interactive_guess import InteractiveGuess2D
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib widget

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from uncertainties import ufloat
%matplotlib inline

In [84]:
x0 = 5
temp = [0.052, 0.051]
x1 = np.array(temp).mean()
x1s = np.array(temp).std()
u1 = ufloat(0.052, 0.01)
print(f'{u1:.2f}, {u1*100:.2f}')

0.05+/-0.01, 5.20+/-1.00


In [93]:
# Allene
t = np.array([0.63, 1.63, 2.63])
rel_inten = np.array([0.05, 0.13, 0.13])
err = np.array([0.01, 0.01, 0.01])

E = 1.9/1000
n = 10

trap_time = n*t*E

x, y = variables('x, y')
A = Parameter('A', min=0, max=1)
k = Parameter('k', min=1, max=50)
model = Model({
    y: A*(1-exp(-k*x))
})
os.chdir(r'D:\SurfDrive\backup_for_surfacePro\Work\C3H3+\Data\datas\full_report_felixFiles\linear_depletion_scan\DATA')

In [97]:
class depletion_model:

    def __init__(self):
        
        width = widgets.IntSlider(min=5,max=20,step=1,value=10)
        height = widgets.IntSlider(min=2,max=10,step=1,value=5)
        dpi = widgets.IntSlider(min=70,max=200,step=1,value=100)

        xlabel = widgets.Text('n*t*E')
        ylabel = widgets.Text('Relative abundance of active isomer')
        title = widgets.Text('Depletion Model')

        accordion_figure = widgets.Accordion(children=[
            widgets.HBox([ widgets.Label(value="Width"), width, widgets.Label(value="Height"), height, widgets.Label(value="DPI"), dpi]),
            widgets.HBox([widgets.Label(value="X-label "), xlabel, widgets.Label(value="Y-Label "), ylabel, widgets.Label(value="Title "), title])
                                               ])
        accordion_figure.set_title(0, 'Size:')
        accordion_figure.set_title(1, 'Labels')

        a = widgets.FloatSlider(min=0,max=1,step=0.01,value=0.2, desciption='A')
        K = widgets.FloatSlider(min=0,max=50,step=0.1,value=5)

        accordion_params = widgets.Accordion(children=[widgets.HBox([widgets.Label(value="A"), a, widgets.Label(value="k"), K])])
        accordion_params.set_title(0, 'Main Parameters:')

        tab_nest = widgets.Tab()
        tab_nest.children = [accordion_params, accordion_figure]
        tab_nest.set_title(0, 'Parameters')
        tab_nest.set_title(1, 'Figure Properties')
        
        self.save_name = widgets.Text('Depletion_Plot')
        save_button = widgets.Button(
            description='Save',
            layout={'width': '300px'}
        )
        
        save_button.on_click(self.save_plot)
        self.print_save = widgets.Label(value = '')
        
        save_layout = widgets.VBox([self.save_name, save_button, self.print_save])
        
        last_layout = widgets.HBox([save_layout, widgets.Label(value = f'Location: {os.getcwd()}')])
        self.out = widgets.interactive_output(self.plot, {'a': a, 'K': K, 'width': width, 'height': height, 'dpi': dpi, 'xlabel': xlabel, 'ylabel': ylabel, 'title': title})
        display(tab_nest, self.out, last_layout)
        
    def plot(self, a, K, width, height, dpi, xlabel, ylabel, title):

        xdata = np.linspace(0, 0.1, 10)
        y = model(x=xdata, A=a, k=K)
        self.fig, ax = plt.subplots(figsize = (width, height), dpi=dpi)
        ax.errorbar(trap_time, rel_inten, yerr = err, fmt='.', label='Exp')
        ax.plot(xdata, y[0], label=f'fit: A:{a:.2f}, k:{K} /J')

        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)
        ax.set_title(title)
        ax.legend()
        ax.grid()
        
    def save_plot(self, event):
        self.fig.savefig(f'./{self.save_name.value}.png')
        
        self.print_save.value='Saved'
        
app = depletion_model()

Output()

In [91]:
fit = Fit(model, x=trap_time, y=rel_inten)
fit_result = fit.execute()
print(fit_result)


Parameter Value        Standard Deviation
A         1.572918e-01 5.162846e-02
k         4.177309e+01 3.092286e+01
Fitting status message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
Number of iterations:   61
Regression Coefficient: 0.8937995197145056

